In [1]:
import dask.dataframe as dd
import dask.bag as da
import os
import json
import dask

from dask.distributed import LocalCluster
cluster = LocalCluster()          # Fully-featured local Dask cluster
client = cluster.get_client()

2024-04-27 17:41:06,359 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 43207747c3ca800c5e48f8d9e8dc3296 initialized by task ('shuffle-transfer-43207747c3ca800c5e48f8d9e8dc3296', 17) executed on worker tcp://127.0.0.1:52119
2024-04-27 17:41:22,050 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 43207747c3ca800c5e48f8d9e8dc3296 deactivated due to stimulus 'task-finished-1714232482.0498052'
2024-04-27 17:42:23,181 - distributed.nanny - WARNING - Restarting worker
2024-04-27 17:42:23,182 - distributed.nanny - WARNING - Restarting worker
2024-04-27 17:42:23,183 - distributed.nanny - WARNING - Restarting worker
2024-04-27 17:42:23,197 - distributed.nanny - WARNING - Restarting worker
2024-04-27 17:42:23,220 - distributed.nanny - WARNING - Restarting worker
2024-04-27 17:42:23,221 - distributed.nanny - WARNING - Restarting worker
2024-04-27 17:42:23,230 - distributed.nanny - WARNING - Restarting worker
2024-04-27 17:42:23,326 - distributed.nanny - WARNING - Restarting

In [2]:
cluster.dashboard_link

'http://127.0.0.1:8787/status'

In [7]:
df = dd.read_csv('gcs://anaconda-public-data/nyc-taxi/csv/2015/green_tripdata_*.csv', 
                 storage_options={'token': 'anon'},
                #parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'], dtype={'tolls_amount': 'float64'}
                 dtype={'Tolls_amount': 'float64',
       'Trip_type ': 'float64'},
                 #parse_dates=['lpep_pickup_datetime', 'Lpep_dropoff_datetime']
                )

In [8]:
df.head()

,lpep_pickup_datetime,Lpep_dropoff_datetime,Store_and_fwd_flag,RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,Trip_distance,...,Extra,MTA_tax,Tip_amount,Tolls_amount,Ehail_fee,Total_amount,Payment_type,Distance_between_service,Time_between_service,Trip_type
0,2015-01-01 00:34:42,2015-01-01 00:38:34,N,1,-73.922592,40.754528,-73.913635,40.765522,1,0.88,...,0.5,0.5,0.0,0.0,NaN,6.3,2,1.0,407,1.0
1,2015-01-01 00:34:46,2015-01-01 00:47:23,N,1,-73.952751,40.677711,-73.981529,40.658978,1,3.08,...,0.5,0.5,0.0,0.0,NaN,13.3,2,0.0,196,1.0
2,2015-01-01 00:34:44,2015-01-01 00:38:15,N,1,-73.843010,40.719055,-73.846581,40.711567,1,0.90,...,0.5,0.5,1.8,0.0,NaN,7.8,1,69.0,3241,1.0
3,2015-01-01 00:34:48,2015-01-01 00:38:08,N,1,-73.860825,40.757793,-73.854042,40.749821,1,0.85,...,0.5,0.5,0.0,0.0,NaN,6.3,2,0.0,299,1.0
4,2015-01-01 00:34:53,2015-01-01 01:09:10,N,1,-73.945183,40.783321,-73.989624,40.765450,1,4.91,...,0.5,0.5,0.0,0.0,NaN,25.8,2,5.0,1287,1.0


2. **Minimum number of passengers**

In [9]:
df['Passenger_count'].min().compute()

0

3. **Maximum number of passengers**

In [10]:
df['Passenger_count'].max().compute()

9

4. **Find the maximum cost for each number of passengers**

In [11]:
max_amout_per_number_of_passengers = df.groupby(['Passenger_count']).agg({'Total_amount': 'max'})
max_amout_per_number_of_passengers

,Total_amount
npartitions=1,
,float64
,...


In [13]:
max_amout_per_number_of_passengers.compute()

,Total_amount
Passenger_count,
2,8011.30
3,2085.80
7,122.00
1,989970.39
4,499.00
5,1239.90
6,874.18
8,383.00
0,351.80
